In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>")) # makes the notebook fill the whole window

import matplotlib.pyplot as plt
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
import matplotlib.patches as patches
from matplotlib import ticker
from matplotlib.collections import PatchCollection
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.patches import Polygon
from sklearn.cluster import DBSCAN
import datetime
import pandas as pd
import time
import shapefile
import os
from os import listdir
from os.path import isfile, isdir, join
import CubicSplineTrack
#(c)2015 Robert Sedgewick, Kevin Wayne, and Robert Dondero
from greatcircle import great_circle
from datetime import date
import shutil
import imageio

from sklearn.cluster import DBSCAN

import warnings
warnings.filterwarnings("ignore")

#Turn off interactive plotting for pyplot
plt.ioff()

ModuleNotFoundError: No module named 'sklearn'

In [2]:
"""
This cell is for config variables ONLY
"""
check_minutes = 60 # minuite var
split_storm = 1 # seperate or no
is_wwlln = 1 # wwlln(1) or glm(0)
initial_data_path = "Data/"
do_all_storms = 1 # 1 for all storms, 0 for specific storms. uses storms_to_run if 0
storms_to_run = []

# ncdf to csv config
glm_ncdf_data_path = "C:/Users/GamerPlayer1/Documents/Digipen/PersonalSVN/Spring22/CSP250/Storm_Data/GLM_setup/Storms/"

In [17]:
"""
Converts multiple .nc files to .csv files
For each input file, read it using GLMDataset
then save only the values we want
"""
import csv
from glmtools.io.glm import GLMDataset

def convert_ncdf_to_csv(file_paths, out_path):
    with open(out_path, 'w', newline='') as out_file:
        writer = csv.writer(out_file)
        writer.writerow(["Date", "Lat", "Long"])

        for in_file_path in file_paths:
            glm_data = GLMDataset(in_file_path).dataset
            groups = glm_data[['group_energy','group_area']]
            group_time = groups['group_time_offset'].values
            group_lon = groups['group_lon'].values
            group_lat = groups['group_lat'].values

            group_size = (glm_data['number_of_groups'].values).size
            group_indx = 0
            while group_indx < group_size:
                writer.writerow([group_time[group_indx], group_lon[group_indx], group_lat[group_indx]])
                group_indx+=1
    return

"""
Testing code for converting local .nc files to .csv files
Gets all files for ONLY one hour, devided into chunks
YOU MUST CHANGE THE PATHS TO GET THIS TO WORK
AND MAKE SURE ALL FOLDERS HAVE BEEN CREATED BEFOREHAND
"""

from os import listdir
def convert_storms():
    if do_all_storms:
        storms_to_run = listdir(glm_ncdf_data_path)
    ncdf_storm_paths = [os.path.join(glm_ncdf_data_path, fn) for fn in storms_to_run]
    csv_storm_paths = [os.path.join(initial_data_path, fn) for fn in storms_to_run]

    curr_storm_indx = 0
    while curr_storm_indx < len(csv_storm_paths):
        if not os.path.exists(csv_storm_paths[curr_storm_indx]):
            os.mkdir(csv_storm_paths[curr_storm_indx])
        
        csv_storm_paths[curr_storm_indx] = os.path.join(csv_storm_paths[curr_storm_indx]+'/', "glm/")

        if not os.path.exists(csv_storm_paths[curr_storm_indx]):
            os.mkdir(csv_storm_paths[curr_storm_indx])
        
        curr_storm_indx+=1
    

    curr_storm_indx = 0
    for ncdf_storm in ncdf_storm_paths:
        ncdf_day_paths = [os.path.join(ncdf_storm, fn) for fn in listdir(ncdf_storm)]
        csv_day_paths = [os.path.join(csv_storm_paths[curr_storm_indx], fn) for fn in listdir(ncdf_storm)]
        

        curr_day_indx = 0
        for ncdf_day in ncdf_day_paths:
            ncdf_hour_paths = [os.path.join(ncdf_day, fn) for fn in listdir(ncdf_day)]
            csv_hour_paths = [os.path.join(csv_day_paths[curr_day_indx], fn) for fn in listdir(ncdf_day)]

            if not os.path.exists(csv_day_paths[curr_day_indx]):
                os.mkdir(csv_day_paths[curr_day_indx])

            curr_hour_indx = 0
            for ncdf_hour in ncdf_hour_paths:
                ncdf_file_paths = [os.path.join(ncdf_hour, fn) for fn in listdir(ncdf_hour)]
                
                if not os.path.exists(csv_hour_paths[curr_hour_indx]):
                    os.mkdir(csv_hour_paths[curr_hour_indx])

                files_per_minuite = 3
                offset = check_minutes*files_per_minuite

                indx = 0
                while indx < 60*files_per_minuite:
                    convert_ncdf_to_csv(ncdf_file_paths[indx:indx+offset], os.path.join(csv_hour_paths[curr_hour_indx]+'/', f'{indx/files_per_minuite}_to_{(indx+offset)/files_per_minuite}.csv'))
                    indx += check_minutes*files_per_minuite
                curr_hour_indx += 1
            curr_day_indx += 1
        curr_storm_indx += 1

convert_storms()

['C:/Users/GamerPlayer1/Documents/Digipen/PersonalSVN/Spring22/CSP250/Storm_Data/GLM_setup/Storms/Dorian\\234\\00\\OR_GLM-L2-LCFA_G16_s20192340000000_e20192340000200_c20192340000226.nc', 'C:/Users/GamerPlayer1/Documents/Digipen/PersonalSVN/Spring22/CSP250/Storm_Data/GLM_setup/Storms/Dorian\\234\\00\\OR_GLM-L2-LCFA_G16_s20192340000200_e20192340000400_c20192340000424.nc', 'C:/Users/GamerPlayer1/Documents/Digipen/PersonalSVN/Spring22/CSP250/Storm_Data/GLM_setup/Storms/Dorian\\234\\00\\OR_GLM-L2-LCFA_G16_s20192340000400_e20192340001000_c20192340001031.nc', 'C:/Users/GamerPlayer1/Documents/Digipen/PersonalSVN/Spring22/CSP250/Storm_Data/GLM_setup/Storms/Dorian\\234\\00\\OR_GLM-L2-LCFA_G16_s20192340001000_e20192340001200_c20192340001230.nc', 'C:/Users/GamerPlayer1/Documents/Digipen/PersonalSVN/Spring22/CSP250/Storm_Data/GLM_setup/Storms/Dorian\\234\\00\\OR_GLM-L2-LCFA_G16_s20192340001200_e20192340001400_c20192340001426.nc', 'C:/Users/GamerPlayer1/Documents/Digipen/PersonalSVN/Spring22/CSP250/

KeyboardInterrupt: 

In [6]:
"""Team WAAAAAAEEEEE Cassie"""

only_dirs = [d for d in listdir(initial_data_path) if isdir(join(initial_data_path, d))] # makes a list of all the folders in Data/
dir_count = len(only_dirs)

WWLLN_Track_Path = [""] * dir_count # the paths of all the track files, in the same order as only_dirs
WWLLN_Locations_Path = [""] * dir_count # the paths of all the lightning locations files, in the same order as only_dirs
Cubic_Spline_Path = [""] * dir_count # the paths of all the cubic spline files, in the same order as only_dirs

Image_Path = [""] * dir_count # Base Image paths
Storm_Names = [""] * dir_count # Base storm names
T_Path = [""] * dir_count

# loop thru all dirs
storms_found = 0
dir_indx = 0
while dir_indx < dir_count:
    
    # this gets all the file paths in a given storms' folder
    only_dirs[dir_indx] = join(initial_data_path, only_dirs[dir_indx])
    glm_path[index] = os.path.join(only_dirs[dir_indx] + '/', 'glm') #Lance code start here 
    if not os.path.exists(glm_path[dir_indx]): 
        glm_path[index] = '' #Lance code end here 
    file_paths = listdir(only_dirs[dir_indx])
    
    # check if the 3 nessisary files exist
    if len(file_paths) < 2:
        only_dirs[dir_indx] = ""
        dir_indx+=1
        continue
    
    #print(only_dirs[dir_indx])
    #print(file_paths)
    
    # for every file in file_paths, sort it into its' respective arrays
    file_indx = 0
    files_found = 0
    Cubic_Found = False
    while file_indx < len(file_paths):
        # creates the files full path
        file_paths[file_indx] = join(only_dirs[dir_indx]+"/", file_paths[file_indx])
        #print(file_paths[file_indx])
        
        # sorts the file path based on the last expected word of a file and it's type
        if file_paths[file_indx].endswith("Trackfile.txt"):
            WWLLN_Track_Path[dir_indx] = file_paths[file_indx]
            files_found+=1
            
            # generates the name of the storm for images
            Storm_Names[dir_indx] = "Hurricane " + WWLLN_Track_Path[dir_indx].rsplit("_", 3)[1]
            
        elif file_paths[file_indx].endswith("Locations.txt"):
            WWLLN_Locations_Path[dir_indx] = file_paths[file_indx]
            files_found+=1
            
        elif file_paths[file_indx].endswith("Trackfile.csv"):
            Cubic_Spline_Path[dir_indx] = file_paths[file_indx]
            Cubic_Found = True
            files_found+=1
        #print(f"File: {file_paths[file_indx]}, Storm indx: {dir_indx}\n")
        file_indx+=1
    
    # Run with Bens' Cubic Spline
    if not Cubic_Found:
        Cubic_Spline_Path[dir_indx] = join(only_dirs[dir_indx]+"/", Storm_Names[dir_indx] + " Cubic Spline Trackfile.csv")
        #print(WWLLN_Track_Path[dir_indx])
        CubicSplineTrack.cubic_spline_trackfile(WWLLN_Track_Path[dir_indx], Cubic_Spline_Path[dir_indx], check_minutes)
        files_found+=1
    
    if files_found != 3:
        print(f"Incorrect files found: {files_found}, {file_paths}\n")
        only_dirs[dir_indx] = ""
        dir_indx+=1
        continue
    
    # makes the path for this index's storm
    Image_Path[dir_indx] = join("./Images/", Storm_Names[dir_indx])
    
    if not os.path.exists(Image_Path[dir_indx]):
        os.mkdir(Image_Path[dir_indx])
    
    T_Path[dir_indx] = Image_Path[dir_indx] + "/T Value"
    Image_Path[dir_indx] = Image_Path[dir_indx] + "/" + Storm_Names[dir_indx]
    
    if split_storm:
        Image_Path[dir_indx] = Image_Path[dir_indx] + " Split " + str(check_minutes) +" Minutes"
    else:
        Image_Path[dir_indx] = Image_Path[dir_indx] + " " + str(check_minutes) +" Minutes"
    
    # if an image folder |dosen't| exist, create a new one with the path at Image_Path[dir_indx]
    if not os.path.exists(Image_Path[dir_indx]):
        os.mkdir(Image_Path[dir_indx])
    
    Image_Path[dir_indx] = Image_Path[dir_indx] + '/' # add a slash for use in the next block, making the actual img paths
    
    if not os.path.exists(T_Path[dir_indx]):
        os.mkdir(T_Path[dir_indx])
        T_Path[dir_indx] = T_Path[dir_indx] + '/'
        if not os.path.exists(T_Path[dir_indx]+"0.0"):
            os.mkdir(T_Path[dir_indx]+"0.0")
        T_value = 1.5
        while T_value < 9:
            if not os.path.exists(T_Path[dir_indx]+str(T_value)):
                os.mkdir(T_Path[dir_indx]+str(T_value))
            T_value+=0.5
    else:
        T_Path[dir_indx] = T_Path[dir_indx] + '/'
    
    #print('\n')
    storms_found+=1
    dir_indx+=1

if not os.path.exists("Images/T Value"):
    os.mkdir("Images/T Value")
    if not os.path.exists("Images/T Value/0.0"):
        os.mkdir("Images/T Value/0.0")
    T_value = 1.5
    while T_value < 9:
        if not os.path.exists("Images/T Value/"+str(T_value)):
            os.mkdir("Images/T Value/"+str(T_value))
        T_value+=0.5

# print for debug
print(f"Storms Found: {storms_found}")
print(Storm_Names)
#print(Image_Path)
#print(WWLLN_Track_Path)
#print(WWLLN_Locations_Path)
#print(Cubic_Spline_Path)

Storms Found: 35
['Hurricane Arthur', 'Hurricane Bertha', 'Hurricane Beta', 'Hurricane Cristobal', 'Hurricane Delta', 'Hurricane Dolly', 'Hurricane Dorian', 'Hurricane Edouard', 'Hurricane Epsilon', 'Hurricane Eta', 'Hurricane Fay', 'Hurricane Five', 'Hurricane Gamma', 'Hurricane Gonzalo', 'Hurricane Hanna', 'Hurricane Iota', 'Hurricane Isaias', 'Hurricane Josephine', 'Hurricane Kyle', 'Hurricane Laura', 'Hurricane Marco', 'Hurricane Nana', 'Hurricane Omar', 'Hurricane Paulette', 'Hurricane Rene', 'Hurricane Sally', 'Hurricane Teddy', 'Hurricane Ten', 'Hurricane Theta', 'Hurricane Twenty', 'Hurricane Twenty-Eig', 'Hurricane Twenty-Nin', 'Hurricane Vicky', 'Hurricane Wilfred', 'Hurricane Zeta']


In [3]:
"""Team WAAAAAEEEEEEEEE Lance"""
def run_circle(row, center_lat, center_lon): 
    lat = (row['Lat'])
    lon = (row['Long'])
    return great_circle(lat, lon, center_lat, center_lon)

In [4]:
"""Team WAAAAAAEEEEEEEEEE Bhargav"""
#Creating intensity table
intensity_values = {
     'T-Number': [1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5, 5.5, 6, 6.5, 7, 7.5, 8, 8.5],
  '1-min Winds': [25, 30, 35, 45, 55, 65, 77, 90, 102, 115, 127, 140, 155, 170, 185],
'Min. Pressure': [0, 1009, 1005, 1000, 994, 987, 979, 970, 960, 948, 935, 921, 906, 890, 873]
}
T_frame = pd.DataFrame(intensity_values)

def Tvalue(wind_speed):
    i=0
    half=0
    current = 0
    prev=0
    after=0
    for i in range(15):
        current = T_frame.loc[i, '1-min Winds']
        if (i == 0):
            prev=20
        else:
            prev = T_frame.loc[(i-1), '1-min Winds']
        
        if(i==14):
            after = 188
            
        else:
            after=T_frame.loc[(i+1), '1-min Winds']

        #if(wind_speed < 25):
            #return 1.5
        
        if(wind_speed > 185):
            return 8.5
        
        else:
            half_prev =current - ((current-prev)/2)
            half_aft = current + ((after-prev)/2)
            if(wind_speed >= half_prev and wind_speed < half_aft):
                return T_frame.loc[i, 'T-Number']
    return 0

In [1]:
"Lance" 
def get_glm(day, hour, start_minute, base_path): 
    return os.path.join(base_path + '/', day + '/', hour + '/', start_minute)
        
    

In [ ]:
"Bhargava's Code"
#Function to get and arrange generated csv file
def get_csv(csv_file_path):
    temp_csv = pd.read_csv(csv_file_path)
    temp_csv= temp_csv.sort_values(["Date"], ascending=True)
    index = pd.Series(range(temp_csv.index.max()+1))
    temp_csv=temp_csv.set_index(index)
    return temp_csv

In [7]:
"""
TODO:
-cite sources (great circle etc.)
-sort output by intensity (Dvorac scale?) 1-5
-statitics over a single storm and multiple storms
-faster file save with PIL?
"""
# for every storm, do the things
"""Team WAAAAAAAEEEEEE Cassie"""
dir_indx = 0
while dir_indx < dir_count:
    # if storm is invalid, skip doin the things
    if only_dirs[dir_indx] == "":
        dir_indx+=1
        continue
    
    """Team Squeem"""
    if is_wwlln:
        # This loads the WWLLN trackfile Jeremy and Natalia give us
        # wind = nautical mph, pressure = millibars
        df = pd.read_csv(WWLLN_Track_Path[dir_indx], header=None, names=["Year","Month","Day","Hour","Lat","Long","Min_Pressure","Max_Winds","Unused"], low_memory=False, sep='\t')
        df = df.drop("Unused", axis=1)#data filtering
        df['Date'] = df.apply(lambda x: pd.to_datetime(f"{int(x['Year'])}-{int(x['Month'])}-{int(x['Day'])}-{int(x['Hour'])}", format="%Y-%m-%d-%H"), axis=1)
    else:
        df = pd.read_csv(WWLLN_Track_Path[dir_indx])
        # other GLM processing
    
    "Team WAAAAAEEEEE Lance"
    #wind pressure stuff 
    #run pressure wind speed function on df for every row, save into array
    Tindex = 0
    Tvalues = df['Max_Winds'].apply(Tvalue)
    #Tvalues = [Tvalue(t) for t in df['Max_Winds'] if isdir(join("Data/", d))]
    #make dataframe 
    dates = df['Date']
    #print(Tvalues)
    #print(df['Max_Winds'])
    if len(Tvalues) != len(dates):
        raise Exception("Lengths differ!")
    #loop through df, for every row, check date 
    df_length = len(dates)
    left_half_dates = [date.min] * df_length
    right_half_dates = [date.min] * df_length
    date_index = 1 
    left_half_dates[0] = date.min
    right_half_dates[df_length - 1] = date.max
    while date_index < df_length: 
        left_half_dates[date_index] = dates.iloc[date_index - 1] + ((dates.iloc[date_index] - dates.iloc[date_index - 1]) / 2)
        date_index += 1
    date_index = 0
    while date_index < df_length - 1: 
        right_half_dates[date_index] = dates.iloc[date_index] + ((dates.iloc[date_index] - dates.iloc[date_index + 1]) / 2)
        date_index += 1
    
    """Team Squeem"""
    if is_wwlln:
        # This loads the WWLLN lighting location data that Jeremy and Natalia give us
        ln = pd.read_csv(WWLLN_Locations_Path[dir_indx], header=None, names=["Year","Month","Day","Hour","Min","Sec","Lat","Long","Dist_East_West","Dist_North_South"], low_memory=False, sep=' ')
        ln['Date'] = ln.apply(lambda x: pd.to_datetime(f"{int(x['Year'])}-{int(x['Month'])}-{int(x['Day'])}-{int(x['Hour'])}-{int(x['Min'])}-{x['Sec']}", format="%Y-%m-%d-%H-%M-%S.%f"), axis=1)#is a data clean
    else:
        ln = pd.read_csv(WWLLN_Locations_Path[dir_indx])
        # other GLM processing

    # This opens the cubic spline trackfiles that Ben creates
    storm_center = pd.read_csv(Cubic_Spline_Path[dir_indx])
    storm_center['Date'] = storm_center['Date'].apply(pd.to_datetime)

    # Genereates the list of dates from the WWLLN trackfile of a (check_minutes) range
    dates = pd.date_range(df['Date'][0], df['Date'][len(df)-1], freq=str(check_minutes)+"T")
    
    buffer = 10
    if split_storm:
        buffer = 6 # affects map zooooooooooooom
    shp_file = "World_Countries__Generalized_-shp/World_Countries__Generalized_" # Map file ???????
    cmap = 'gist_ncar' # dunno WHAT this is

    # Counter and iteration
    for index, date in enumerate(dates): # this loop makes one image for every (check_minutes) amount of time
        starttime = time.perf_counter() # speeed timer start
        center = storm_center[storm_center['Date'] == date] # storm center at this time frame
        """Team WAAAAAAEEEEE Cassie and Lance"""
        if split_storm:
            ln_curr_timeframe = ln[(ln['Date'] >= date) & (ln['Date'] < date + datetime.timedelta(minutes=check_minutes))]
            #print(ln_curr_timeframe)
            if len(ln_curr_timeframe) != 0:
                ln_curr_timeframe['Distance'] = ln_curr_timeframe.apply(run_circle, axis = 1, args = ((center['Lat']), (center['Long'])))
                #print(ln_curr_timeframe)
                subset = ln_curr_timeframe[(ln_curr_timeframe['Distance'] <= 100 / 1.852) | ((ln_curr_timeframe['Distance'] >= 200 / 1.852) & (ln_curr_timeframe['Distance'] <= 400 / 1.852))]
            else:
                subset = ln_curr_timeframe
        else:
            # Select just the lightning locations that are within (check_minutes) minutes
            subset = ln[(ln['Date'] >= date) & (ln['Date'] < date + datetime.timedelta(minutes=check_minutes))]
        
        """Team WAAAAAAEEEEEE Cassie"""
        if date >= right_half_dates[Tindex]:
            Tindex += 1
        stormTval = Tvalues[Tindex]
        
        # if lightning exists, do dbscan on it
        if len(subset != 0):
            dbscan = DBSCAN(eps=.2, min_samples=3)
            subset['Labels'] = dbscan.fit_predict(subset[['Long','Lat']])
        
        """Team Squeem"""
        # Setup the figure
        fig, ax = plt.subplots(figsize=(15,15))
        
        # this is for the coastline
        # This reads the shapefile, extracts each shape, creates a polygon, and adds it to the list
        sf = shapefile.Reader(shp_file)
        shapes = sf.shapes()
        Nshp = len(shapes)
        ptchs = []
        for nshp in range(Nshp):
            pts = np.array(shapes[nshp].points)
            prt = shapes[nshp].parts
            par = list(prt) + [pts.shape[0]]
            for pij in range(len(prt)):
                ptchs.append(Polygon(pts[par[pij]:par[pij+1]]))
        # Every every polygon from the shapefile to figure
        ax.add_collection(PatchCollection(ptchs, facecolor= '#838688', edgecolor='k', linewidths=1., zorder=2))

        # This adds the water color
        ax.add_patch(mpl.patches.Rectangle((-180,-89),360,180,color='#a6cae0'))

        # This connects Antarcitca to the bottom of the image
        ax.add_patch(mpl.patches.Rectangle((-179.9,-89.9),360,2,color='#838688',zorder=3))

        # These are more general image settings
        plt.title(Storm_Names[dir_indx]+" T"+str(stormTval)+' '+str(date), fontdict = {'fontsize' : 40})

        #edges = (-180,180,-90,90) # Left, Right, Bottom, Top | Set this if you want to change the map scale
        edges = (center['Long'].values[0] - buffer, center['Long'].values[0] + buffer, center['Lat'].values[0] - buffer, center['Lat'].values[0] + buffer) # <--------------------------This affects map zoooooooooom

        # setting up img stuff, prob shouldn't be touched
        xlim = np.append(np.arange(edges[0], edges[1], step=2.5), edges[1])
        ylim = np.append(np.arange(edges[2], edges[3], step=2.5), edges[3])
        ax.set_xticks(xlim)
        ax.set_yticks(ylim)
        ax.set_xlim(edges[0], edges[1])
        ax.set_ylim(edges[2], edges[3])
        ax.xaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
        ax.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.2f'))
        plt.rc('xtick', labelsize=30)
        plt.rc('ytick', labelsize=30)
        
        """Team WAAAAAEEEEEE Cassie"""
        # if lightning exists, put it in the image
        if len(subset) != 0:
            plt.scatter(subset['Long'], subset['Lat'], c=subset['Labels'], cmap=cmap, zorder=4) # plots the dbscan data
        
        """Team Squeem"""
        #plt.scatter(subset['Long'],subset['Lat'],s=100,c="yellow",edgecolors='black',zorder=5) # <- old plot func for non-grouped data
        plt.scatter(center['Long'], center['Lat'], s=140, c="red", edgecolors='black' ,zorder=5, alpha=0.5) # plots the center of the storm

        # save and close the image
        plt.savefig(Image_Path[dir_indx] + str(index)+".png", bbox_inches='tight', pad_inches=.4)
        if split_storm:
            shutil.copy(Image_Path[dir_indx] + str(index)+".png", T_Path[dir_indx] + str(stormTval) + '/' + Storm_Names[dir_indx] + ' ' + str(index)+".png")
            #plt.savefig(T_Path[dir_indx] + str(stormTval) + '/' + Storm_Names[dir_indx] + ' ' + str(index), bbox_inches='tight', pad_inches=.4)
            shutil.copy(Image_Path[dir_indx] + str(index)+".png", "Images/T Value/" + str(stormTval) + '/' + Storm_Names[dir_indx] + ' ' + str(index)+".png")
            #plt.savefig("Images/T Value/" + str(stormTval) + '/' + Storm_Names[dir_indx] + ' ' + str(index), bbox_inches='tight', pad_inches=.4)
        plt.close('all')

        # Shows roughly how long we have left
        taken = time.perf_counter() - starttime
        print(f"Storm #{dir_indx}: {Storm_Names[dir_indx]}, T{stormTval}\tPercent Done: {(index + 1) / len(dates) * 100 : .2f}%\tTime taken: {taken : .2f} seconds\tEst time remaining: {datetime.timedelta(seconds=(len(dates) - (index + 1)) * taken)} {len(subset)}", end='\r')
    dir_indx+=1

In [8]:
"""Team WAAAAAAEEEEE Cassie"""
"""This makes the GIFs"""
dir_indx = 0
while dir_indx < dir_count:
    with imageio.get_writer(Image_Path[dir_indx]+'All.gif', mode='I') as writer:
        Images = [f for f in listdir(Image_Path[dir_indx]) if isfile(join(Image_Path[dir_indx], f)) and f.endswith(".png")]
        img_cnt = len(Images)
        img_indx = 0
        while img_indx < img_cnt:
            image = imageio.imread(Image_Path[dir_indx] + str(img_indx) + ".png")
            writer.append_data(image)
            img_indx+=1
    dir_indx+=1

In [12]:
"""
Useful for knowing what columns are in the dataset for coding
appears 'printed' below box, has buttons that can be clicked to show/hide info
"""
from glmtools.io.glm import GLMDataset
glm_data = GLMDataset("C:/Users/GamerPlayer1/Documents/Digipen/PersonalSVN/Spring22/CSP250/Storm_Data/GLM_setup/Dorian/234/00/OR_GLM-L2-LCFA_G16_s20192340000000_e20192340000200_c20192340000226.nc").dataset
glm_data

INFO:glmtools.io.glm:Inferring lightning ellipsoid from GLM product time
INFO:glmtools.io.glm:Using lightning ellipsoid rev 1
DEBUG:glmtools.io.glm:New flash area units: km2
DEBUG:glmtools.io.glm:New group area units: km2
DEBUG:glmtools.io.glm:Setting zero-energy events to 7.629849913322491e-16


<xarray.Dataset>
Dimensions:                                 (number_of_events: 25335, number_of_field_of_view_bounds: 2, number_of_flashes: 588, number_of_groups: 9422, number_of_time_bounds: 2, number_of_wavelength_bounds: 2)
Coordinates: (12/21)
    event_id                                (number_of_events) uint32 1612830...
    event_time_offset                       (number_of_events) datetime64[ns] ...
    event_lat                               (number_of_events) float32 -4.533...
    event_lon                               (number_of_events) float32 -60.26...
    event_parent_group_id                   (number_of_events) uint32 6359961...
    group_id                                (number_of_groups) uint32 6359961...
    ...                                      ...
    product_time                            datetime64[ns] 2019-08-22
    lightning_wavelength                    float32 ...
    group_time_threshold                    float32 ...
    flash_time_threshold                    float32 ...
    lat_field_of_view                       float32 ...
    lon_field_of_view                       float32 -75.0
Dimensions without coordinates: number_of_events, number_of_field_of_view_bounds, number_of_flashes, number_of_groups, number_of_time_bounds, number_of_wavelength_bounds
Data variables: (12/37)
    event_energy                            (number_of_events) float32 1.526e...
    group_frame_time_offset                 (number_of_groups) datetime64[ns] ...
    group_area                              (number_of_groups) float32 136.3 ...
    group_energy                            (number_of_groups) float32 3.052e...
    group_quality_flag                      (number_of_groups) float32 ...
    flash_frame_time_offset_of_first_event  (number_of_flashes) datetime64[ns] ...
    ...                                      ...
    event_x                                 (number_of_events) float32 0.045 ...
    event_y                                 (number_of_events) float32 -0.013...
    group_x                                 (number_of_groups) float32 0.0449...
    group_y                                 (number_of_groups) float32 -0.013...
    flash_x                                 (number_of_flashes) float32 0.044...
    flash_y                                 (number_of_flashes) float32 -0.01...
Attributes: (12/29)
    production_site:           WCDAS
    featureType:               point
    dataset_name:              OR_GLM-L2-LCFA_G16_s20192340000000_e2019234000...
    naming_authority:          gov.nesdis.noaa
    Conventions:               CF-1.7
    institution:               DOC/NOAA/NESDIS> U.S. Department of Commerce, ...
    ...                        ...
    time_coverage_start:       2019-08-22T00:00:00.0Z
    time_coverage_end:         2019-08-22T00:00:20.0Z
    production_data_source:    Realtime
    production_environment:    OE
    LUT_Filenames:             GLM_CALINR_AllFilters(FM1E_CDRL79RevK_PR_08_01...
    id:                        39d56f63-be20-4a43-a663-4974284f8cfa

In [2]:
"""
Generic testing code for how to interact with the dataset
"""
from glmtools.io.glm import GLMDataset
glm_data = GLMDataset("C:/Users/GamerPlayer1/Documents/Digipen/PersonalSVN/Spring22/CSP250/Storm_Data/GLM_setup/Dorian/234/00/OR_GLM-L2-LCFA_G16_s20192340000000_e20192340000200_c20192340000226.nc").dataset
groups = glm_data[['group_energy','group_area']]
flashes = glm_data[['flash_energy', 'flash_area']]
# remove unused data (optional)
# groups = groups.drop(['group_parent_flash_id','lightning_wavelength','product_time','group_time_threshold','flash_time_threshold','lat_field_of_view','lon_field_of_view'])
# get the lattitude and longitude values
group_lon, group_lat = groups['group_lon'].values, groups['group_lat'].values
flash_lon, flash_lat = flashes['flash_lon'].values, flashes['flash_lat'].values

(glm_data['number_of_groups'].values).size

9422

In [4]:
#Testing the function with file path to csv file
test_csv = get_csv("C:/Users/bharg/Documents/DigiPen/Spring 2022/CSP 250/Project/Data/GLM_setup/Test/001/csv/00/0.0_to_10.0.csv")
test_csv.head()

,Date,Lat,Long
0,2020-12-31T23:59:59.451820374,-68.219690,-20.832464
1,2020-12-31T23:59:59.459831238,-68.219690,-20.832394
2,2020-12-31T23:59:59.492256642,-48.615196,-14.822459
3,2020-12-31T23:59:59.496071339,-48.615192,-14.822376
4,2020-12-31T23:59:59.520104409,-48.615240,-14.822432
